In [3]:
"""

Please read!
This model takes in one-hot encoded molecules. This is essentially taking a String representation of a molecule and breaking it down into individual characters (characterized input). 
It processes a sequence of vector X, and taking as input each item x[i] in the sequence.

The outputed molecules must be determined if chemically valid or not. In the future, we can check the validity by comparing molecules to the original SMILES 
input used for training. Once we find the common physiochemical features of the data, we can calculate the common physiochemical features for the data. In 
addition, executing a Principal Component Analysis (PCA) on the features, and transform the newly generated molecules accordingly. 

Model uses Tensorflow backed with Keras

Comment out specific parts of the code depending on the use, between training, saving checkpoints, sampling, etc.

"""

'\n\nPlease read!\nThis model takes in one-hot encoded molecules. This is essentially taking a String representation of a molecule and breaking it down into individual characters (characterized input). \nIt processes a sequence of vector X, and taking as input each item x[i] in the sequence.\n\nThe outputed molecules must be determined if chemically valid or not. In the future, we can check the validity by comparing molecules to the original SMILES \ninput used for training. Once we find the common physiochemical features of the data, we can calculate the common physiochemical features for the data. In \naddition, executing a Principal Component Analysis (PCA) on the features, and transform the newly generated molecules accordingly. \n\nModel uses Tensorflow backed with Keras\n\nComment out specific parts of the code depending on the use, between training, saving checkpoints, sampling, etc.\n\n'

In [4]:
import sys
import numpy as np
from numpy.testing import assert_allclose
from keras.models import Sequential, load_model
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

/usr/local/lib/python3.6/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [5]:
# Opening files, extracting data, and automatically closing them (SMILES strings are conjoined together with the "\n" metatag)
filename = '100k_rndm_zinc_drugs_clean.txt'
with open(filename) as f:
	# f = [next(filename) for x in range(10000)]
    	raw_text = "\n".join(line.strip() for line in f)


In [6]:
# creating mapping for each char to integer, also mapping for the \n (new line) is manually inserted into the dictionaries.
unique_chars = sorted(list(set(raw_text)))
# maps each unique character as int
char_to_int = dict((c, i) for i, c in enumerate(unique_chars))
# manually updates \n
char_to_int.update({-1 : "\n"})

# int to char dictionary
int_to_char = dict((i, c) for i, c in enumerate(unique_chars))
int_to_char.update({"\n" : -1})


In [7]:
# how many unique characters
mapping_size = len(char_to_int)
reverse_mapping_size = len(int_to_char)
print ("Size of the character to integer dictionary is: ", mapping_size)
print ("Size of the integer to character dictionary is: ", reverse_mapping_size)


Size of the character to integer dictionary is:  35
Size of the integer to character dictionary is:  35


In [8]:
assert mapping_size == reverse_mapping_size

In [9]:
# Summarize the loaded data to provide lengths for preparing datasets
n_chars = len(raw_text)
n_vocab = len(unique_chars)

print ("Total number of characters in the file is: ", n_chars)

# Preparring datasets by matching the dataset lengths (dataX will be the SMILES strings and dataY will be individual characters in the SMILE string)
seq_length = 137
dataX = []
dataY = []



Total number of characters in the file is:  4530479


In [10]:
for i in range(0, n_chars - seq_length, 1):
	seq_in = raw_text[i:i + seq_length]
	seq_out = raw_text[i + seq_length]
	dataX.append([char_to_int[char] for char in seq_in])
	dataY.append(char_to_int[seq_out])

In [11]:
n_patterns = len(dataX)

In [12]:
# re shape input sequence X (using numpy)to be [samples, time steps, physiochemical features], input format for recurrent models
X = np.reshape(dataX, (n_patterns, seq_length, 1))

# Normalize the integers in X by dividing by the number of unique SMILES characters (a.k.a vocabulary)
X = X / float(n_vocab)

# One-hot encode the output variable (so that they can be used to generate new SMILES after training)
Y = np_utils.to_categorical(dataY)

In [13]:
"""CREATING THE LSTM MODEL"""

# Create the model (simple 2 layer LSTM)
model = Sequential()
model.add(LSTM(128, input_shape=(X.shape[1], X.shape[2]), return_sequences = True))
model.add(Dropout(0.25))
model.add(LSTM(256, return_sequences = True))
model.add(Dropout(0.25))
model.add(LSTM(512, return_sequences = True))
model.add(Dropout(0.25))
model.add(LSTM(256, return_sequences = True))
model.add(Dropout(0.25))
model.add(LSTM(128))
model.add(Dropout(0.25))
model.add(Dense(Y.shape[1], activation='softmax'))


In [14]:
print (model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 137, 128)          66560     
_________________________________________________________________
dropout_1 (Dropout)          (None, 137, 128)          0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 137, 256)          394240    
_________________________________________________________________
dropout_2 (Dropout)          (None, 137, 256)          0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 137, 512)          1574912   
_________________________________________________________________
dropout_3 (Dropout)          (None, 137, 512)          0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 137, 256)          787456    
__________

In [15]:
# Compile the model
model.compile(loss = 'categorical_crossentropy', optimizer='adam')


In [16]:
# # Define checkpoints (used to save the weights at each epoch, so that the model doesn't need to be retrained)
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor = 'loss', verbose = 1, save_best_only = True, mode = 'min')
callbacks_list = [checkpoint]

# # Fit the model
model.fit(X, Y, epochs = 5, batch_size = 512, callbacks = callbacks_list)



Epoch 1/5
4530342/4530342 [==============================] - 14907s 3ms/step - loss: 2.6472

Epoch 00001: loss improved from inf to 2.64723, saving model to weights-improvement-01-2.6472.hdf5
Epoch 2/5
4530342/4530342 [==============================] - 14883s 3ms/step - loss: 2.6436

Epoch 00002: loss improved from 2.64723 to 2.64361, saving model to weights-improvement-02-2.6436.hdf5
Epoch 3/5
1826304/4530342 [===========>..................] - ETA: 2:28:05 - loss: 2.6432

KeyboardInterrupt: 

In [ ]:
# """TO TRAIN FROM SAVED CHECKPOINT"""
# # Load weights
# model.load_weights("weights-improvement-75-1.8144.hdf5")

# # load the model
# new_model = load_model("model.h5")
# assert_allclose(model.predict(x_train),
#                 new_model.predict(x_train),
#                 1e-5)

# # fit the model
# checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
# callbacks_list = [checkpoint]
# new_model.fit(x_train, y_train, epochs = 100, batch_size = 64, callbacks = callbacks_list)


In [18]:
"""GENERATING NEW SMILES"""

# Load the pre-trained network weights
filename = "weights-improvement-02-2.6436.hdf5"
model.load_weights(filename)
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam')

In [19]:
# Pick a random seed from the SMILES strings
start = np.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print ("Seed:")
print ("\"", ''.join([int_to_char[value] for value in pattern]), "\"")


Seed:
" C(=O)Nc3ccc(C(N)=O)cc3)nnn2-c2ccc(F)cc2)n1
COc1ccc([C@@H]2CC(c3ccccc3)=NN2C(=O)CNc2ccccc2)cc1
Cc1ccncc1NC(=O)N(C)[C@@H](C)c1ccncc1
C[C@H] "


In [21]:
# Generate specified number of characters in range
for i in range(137):
	x = np.reshape(pattern, (1, len(pattern), 1))
	prediction = model.predict(x, verbose=0)
	index = np.argmax(prediction)
	result = int_to_char[index]
	seq_in = [int_to_char[value] for value in pattern]
	sys.stdout.write(result)
	pattern.append(index)
	pattern = pattern[1:len(pattern)]
print ("\nDone.")



ccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccc
Done.
